In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
import catboost

In [4]:
data = pd.read_csv("train.csv")
y = np.array(data.iloc[:, 1:3])
y

array([[77.59 ,  6.881],
       [78.71 , 11.057],
       [78.85 ,  8.596],
       ...,
       [32.313,  2.578],
       [33.367,  2.114],
       [32.213,  2.479]])

In [31]:
data = pd.read_csv("train.csv")

X = np.array(data.iloc[:, 3:])
y = np.array(data.iloc[:, 1:3])
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.80)

params = {
    'learning_rate' : [0.0001, 0.1, 0.01, 0.001, 1],
    'n_estimators':[10, 20, 80, 100],
    'max_depth' : [1, 2, 4, 10, 15],
    'reg_lambda' : [0, 2, 10],
    'gamma' : [0, 0.01, 0.25, 1]
}
clf = XGBRegressor(seed = 42)
model = RandomizedSearchCV(estimator=clf, param_distributions=params, cv=5, n_jobs = -1, random_state=42)
model.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          m...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None,
                                          reg_alpha=None, reg_lambda=None, ...),
                   n_jobs=-1,
                   param_distributions={'gamma': [0, 0.01, 0.25, 1],
                                        'learning_rate': [0.0001, 0.1, 0.01,
                                                          0.001, 1],
                                        'max_depth': [1, 2, 4, 10, 15],
                                        'n_estimators': [10, 20, 80, 100],
                                        'reg_lambda': [0, 2, 10]},
                   random_state=42)

In [32]:
model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.25, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='', learning_rate=1,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=2,
             max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=80, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=10, ...)

In [12]:
test_data = pd.read_csv("dummy_test.csv")
X_test = np.array(test_data.iloc[:, 3:])
y_test = np.array(test_data.iloc[:,1:3])
print(y_test)

[[71.327  8.801]
 [72.317  5.536]
 [74.44   4.574]
 ...
 [ 8.782 17.423]
 [ 9.678 16.017]
 [ 9.579 14.75 ]]


In [54]:
print(model.best_params_)

{'reg_lambda': 10, 'n_estimators': 80, 'max_depth': 2, 'learning_rate': 1, 'gamma': 0.25}


In [78]:
import time
from lightgbm import LGBMRegressor
start = time.perf_counter()
lmodel = LGBMRegressor(n_estimators=100, learning_rate=0.3, random_state = 42, n_jobs=-1)
lmodel.fit(X, y)
print("time elapsed: {} seconds".format(time.perf_counter() - start))

ValueError: y should be a 1d array, got an array of shape (20000, 2) instead.

In [33]:
import time
# print(model.best_params_)
# xgbmodel = XGBRegressor(
#                     reg_lambda = 10,
#                     n_estimators = 100,
#                     max_depth =  10,
#                     learning_rate =  1e-4,
#                     gamma =  0.3)
xgbmodel = model.best_estimator_
start = time.perf_counter()
# xgbmodel = XGBRegressor(learning_rate=0.9, n_estimators = 100, max_depth= 10, gamma = 0.25)
xgbmodel.fit(X_train, y_train)
print("time elapsed: {} seconds".format(time.perf_counter() - start))

time elapsed: 0.4303812019970792 seconds


In [73]:
import time
start = time.perf_counter()
rfmodel = RandomForestRegressor(n_estimators=5,  
                                max_depth=2,
                                min_impurity_decrease=1e-6,
                                min_samples_split=0.02,
                                random_state=42, n_jobs = -1)
rfmodel.fit(X, y)
print("time elapsed: {} seconds".format(time.perf_counter() - start))

time elapsed: 0.024755557000389672 seconds


In [34]:
train_preds = xgbmodel.predict(X_train)
test_preds = xgbmodel.predict(X_test)
test_preds = np.transpose(test_preds)
train_preds = np.transpose(train_preds)
y_train = np.transpose(y_train)
y_test = np.transpose(y_test)
print("The MSE Loss for the test samples are {} and {}".format(mean_absolute_error(test_preds[0], y_test[0]), mean_absolute_error(test_preds[1], y_test[1])))
print("The MSE Loss for the train samples are {} and {}".format(mean_absolute_error(train_preds[0], y_train[0]), mean_absolute_error(train_preds[1], y_train[1])))

The MSE Loss for the test samples are 5.13201984669745 and 4.6477414028944075
The MSE Loss for the train samples are 4.9560677077652215 and 4.540054562207393


In [39]:
train_preds = xgbmodel.predict(X)
test_preds = xgbmodel.predict(X_test)
test_preds = np.transpose(test_preds)
train_preds = np.transpose(train_preds)
y_train = np.transpose(y)
y_test = np.transpose(y_test)
print("The MSE Loss for the test samples are {} and {}".format(mean_absolute_error(test_preds[0], y_test[0]), mean_absolute_error(test_preds[1], y_test[1])))
print("The MSE Loss for the train samples are {} and {}".format(mean_absolute_error(train_preds[0], y_train[0]), mean_absolute_error(train_preds[1], y_train[1])))

NameError: name 'xgbmodel' is not defined

In [62]:
train_preds.shape

(2, 20000)